## Some work testing the bettor class

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random

import sys
path_to_working_module = r'C:\Users\greg\Documents\NFL\survivor_sim'
sys.path.append(path_to_working_module)

from survivor_sim.bettor import bettor
from survivor_sim.tournament import tournament

import pandas as pd

In [3]:
path = r'C:\Users\greg\Documents\NFL\survivor_sim\processed_data'

spread_scores = pd.read_csv(path + '\\spreadspoke_scores_processed.csv')
spread_scores.head()


,Unnamed: 0,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,...,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_home_id,team_away_id,winner,favorite_won,team_underdog_id
0,0,09/01/1979,1979,1,False,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,TB,...,False,79.0,9.0,87.0,NaN,TB,DET,TB,True,DET
1,1,11/23/1980,1980,12,False,Tampa Bay Buccaneers,10.0,24.0,Detroit Lions,TB,...,False,69.0,9.0,77.0,NaN,TB,DET,DET,False,DET
2,2,10/04/1981,1981,5,False,Tampa Bay Buccaneers,28.0,10.0,Detroit Lions,TB,...,False,76.0,9.0,76.0,NaN,TB,DET,TB,True,DET
3,3,12/26/1982,1982,8,False,Tampa Bay Buccaneers,23.0,21.0,Detroit Lions,TB,...,False,73.0,11.0,72.0,NaN,TB,DET,TB,True,DET
4,4,09/04/1983,1983,1,False,Tampa Bay Buccaneers,0.0,11.0,Detroit Lions,TB,...,False,82.0,7.0,83.0,NaN,TB,DET,DET,False,DET


In [4]:
# isolate 2018 week 1 for testing
boollist  = (spread_scores.schedule_season==2018)&(spread_scores.schedule_week=='1')

games = spread_scores[['schedule_season', 'schedule_week', 
               'team_home_id', 'team_away_id','team_favorite_id', 
               'team_underdog_id', 'spread_favorite']][boollist]

## Bettor Class testing

In [5]:
# instanciate a bettor class
b = bettor()
b.picked

[]

In [6]:
# test the run_model method
bet_strength = b.run_model(2018, 1, games)
bet_strength.sort_values(by='bet_strength', ascending = False).head()

,team,bet_strength
9664,NO,10.0
5421,BAL,7.5
6048,DET,7.0
472,LAR,6.5
2018,GB,6.5


In [7]:
# test the make_pick method
picks = []
for i in range (1000):
    b.top_n = 3
    b.picked = []
    picks.append(b.make_pick(2018, 1, games))
picks_df = pd.DataFrame(picks)

In [8]:
picks_df[0].value_counts()

NO     394
BAL    322
DET    284
Name: 0, dtype: int64

In [9]:
team_list = ['NO', 'BAL', 'DET']
for team in team_list:
    bs = bet_strength[bet_strength.team == team].bet_strength
    tbs = bet_strength[bet_strength.team.isin(team_list)].sum().bet_strength
    pct = bs/ tbs
    print(team,pct)

NO 9664    0.408163
Name: bet_strength, dtype: float64
BAL 5421    0.306122
Name: bet_strength, dtype: float64
DET 6048    0.285714
Name: bet_strength, dtype: float64


### The bettor class appears to be working as intended

# Run some tournaments

In [17]:
# instanciate 5 bettors with different strategies
bettors = []
strategies = []
for i in range(5):
    b = bettor(top_n = i+1)
    bettors.append(b)
    strategies.append('top_n='+str(i+1))


# dataframe
all_results = pd.DataFrame(columns = ['strategy', 'season', 'weeks_survived'])

# run the seasons
for season in range (1998, 2019):
    t = tournament(bettors, strategies, season, spread_scores)
    t.run()
    all_results = all_results.append(t.results, sort = False)
    all_results['tournament_obj'] = t

all_results.head()

,strategy,season,weeks_survived,winnings,tournament_obj
<survivor_sim.bettor.bettor object at 0x0000019672087C18>,top_n=1,1998.0,10.0,1.0,<survivor_sim.tournament.tournament object at ...
<survivor_sim.bettor.bettor object at 0x0000019672087A20>,top_n=2,1998.0,4.0,0.0,<survivor_sim.tournament.tournament object at ...
<survivor_sim.bettor.bettor object at 0x0000019672087518>,top_n=3,1998.0,7.0,0.0,<survivor_sim.tournament.tournament object at ...
<survivor_sim.bettor.bettor object at 0x00000196720879E8>,top_n=4,1998.0,4.0,0.0,<survivor_sim.tournament.tournament object at ...
<survivor_sim.bettor.bettor object at 0x0000019672087C88>,top_n=5,1998.0,7.0,0.0,<survivor_sim.tournament.tournament object at ...


In [31]:
# summarize the results
print('The results after', all_results.season.max() - all_results.season.min() + 1, 'seasons')
all_results.groupby(by = 'strategy').agg({'weeks_survived':['mean', 'median', 'max'], 'winnings': 'mean'})

The results after 21.0 seasons


weeks_survived               winnings
                   mean median   max      mean
strategy                                      
top_n=1        4.428571    4.0  16.0  0.350794
top_n=2        3.238095    3.0  13.0  0.112698
top_n=3        3.238095    3.0   9.0  0.104762
top_n=4        3.047619    1.0  15.0  0.215873
top_n=5        2.952381    2.0  11.0  0.215873

In [29]:
# you might wonder what happened in season 2000.  access the tournament object
all_results[all_results.season == 2000].tournament_obj[0].picks

,strategy,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
bettor,,,,,,,,,,,,,,,,,,
<survivor_sim.bettor.bettor object at 0x0000019672087C18>,top_n=1,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
<survivor_sim.bettor.bettor object at 0x0000019672087A20>,top_n=2,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
<survivor_sim.bettor.bettor object at 0x0000019672087518>,top_n=3,DET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
<survivor_sim.bettor.bettor object at 0x00000196720879E8>,top_n=4,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
<survivor_sim.bettor.bettor object at 0x0000019672087C88>,top_n=5,GB,SF,NE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## modify the child class of bettor

In [36]:
class no_favorite(bettor):
    def run_model(self, season, week, data):
        '''
        revise the bettor base class to never pick the favorite 
        '''
        model_results = pd.DataFrame()
        model_results['team'] = data.team_favorite_id
        model_results['bet_strength'] = -data.spread_favorite
        # make the spread_favorite go the 0 for the best bet_strength
        model_results.loc[model_results.bet_strength == model_results.bet_strength.max(), 
                          'bet_strength'] = 0
        return model_results

    

In [37]:
b = no_favorite()
b.picked

[]

In [38]:
# test the run_model method
bet_strength = b.run_model(2018, 1, games)
bet_strength.sort_values(by='bet_strength', ascending = False)

,team,bet_strength
5421,BAL,7.5
6048,DET,7.0
472,LAR,6.5
2018,GB,6.5
6928,MIN,6.0
9159,NE,6.0
2452,LAC,3.5
3312,PIT,3.5
7818,DEN,3.0
7560,CAR,2.5


Try running a tournament with 9 bettor selecting the top pick, and 1 bettor always avoiding the favorite

In [61]:
def run_seasons(bettors, strategies, start = 1999, finish = 2019):
    # dataframe
    all_results = pd.DataFrame(columns = ['tournament','strategy', 'season', 'weeks_survived'])

    # run the seasons
    for season in range (start, finish):
        t = tournament(bettors, strategies, season, spread_scores)
        t.run()
        results = t.results
        results['tournament_obj'] = t
        all_results = all_results.append(results, sort = False)
        

    return all_results

In [62]:
# instanciate the bettors
bettors = []
strategies = []
for i in range(9):
    b = bettor()
    bettors.append(b)
    strategies.append('top_n=1')
b = no_favorite()
bettors.append(b)
strategies.append('no_fav')

In [63]:
all_results = run_seasons(bettors, strategies)

In [71]:
# summarize the results
print('The results after', all_results.season.max() - all_results.season.min() + 1, 'seasons')
all_results.groupby(by = 'strategy').agg({'weeks_survived':['mean', 'median', 'max'], 'winnings': 'mean'})

The results after 20.0 seasons


weeks_survived               winnings
                   mean median   max      mean
strategy                                      
no_fav         3.150000    3.5  11.0  0.420000
top_n=1        3.872222    4.0  16.0  0.064444

In [70]:
all_results

,tournament,strategy,season,weeks_survived,winnings,tournament_obj
<survivor_sim.bettor.bettor object at 0x0000019672794BE0>,NaN,top_n=1,1999.0,0.0,0.000000,<survivor_sim.tournament.tournament object at ...
<survivor_sim.bettor.bettor object at 0x00000196727948D0>,NaN,top_n=1,1999.0,0.0,0.000000,<survivor_sim.tournament.tournament object at ...
<survivor_sim.bettor.bettor object at 0x0000019672794D30>,NaN,top_n=1,1999.0,0.0,0.000000,<survivor_sim.tournament.tournament object at ...
<survivor_sim.bettor.bettor object at 0x0000019672794D68>,NaN,top_n=1,1999.0,0.0,0.000000,<survivor_sim.tournament.tournament object at ...
<survivor_sim.bettor.bettor object at 0x0000019672794940>,NaN,top_n=1,1999.0,0.0,0.000000,<survivor_sim.tournament.tournament object at ...
<survivor_sim.bettor.bettor object at 0x0000019672794E48>,NaN,top_n=1,1999.0,0.0,0.000000,<survivor_sim.tournament.tournament object at ...
<survivor_sim.bettor.bettor object at 0x0000019672794AC8>,NaN,top_n=1,1999.0,0.0,0.000000,<survivor_sim.tournament.tournament object at ...
<survivor_sim.bettor.bettor object at 0x0000019672794358>,NaN,top_n=1,1999.0,0.0,0.000000,<survivor_sim.tournament.tournament object at ...
<survivor_sim.bettor.bettor object at 0x0000019672794588>,NaN,top_n=1,1999.0,0.0,0.000000,<survivor_sim.tournament.tournament object at ...
<__main__.no_favorite object at 0x0000019672794A90>,NaN,no_fav,1999.0,4.0,1.000000,<survivor_sim.tournament.tournament object at ...


In [67]:
# you might wonder what happened in a season. 
season = 1999
t = all_results[all_results.season == season].tournament_obj[0]

In [68]:
all_results[all_results.season == season].tournament_obj[0].season

1999

In [69]:
t.picks

,strategy,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
bettor,,,,,,,,,,,,,,,,,,,
<survivor_sim.bettor.bettor object at 0x0000019672794BE0>,top_n=1,SEA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
<survivor_sim.bettor.bettor object at 0x00000196727948D0>,top_n=1,SEA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
<survivor_sim.bettor.bettor object at 0x0000019672794D30>,top_n=1,SEA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
<survivor_sim.bettor.bettor object at 0x0000019672794D68>,top_n=1,SEA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
<survivor_sim.bettor.bettor object at 0x0000019672794940>,top_n=1,SEA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
<survivor_sim.bettor.bettor object at 0x0000019672794E48>,top_n=1,SEA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
<survivor_sim.bettor.bettor object at 0x0000019672794AC8>,top_n=1,SEA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
<survivor_sim.bettor.bettor object at 0x0000019672794358>,top_n=1,SEA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
<survivor_sim.bettor.bettor object at 0x0000019672794588>,top_n=1,SEA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
